
# Watershed Dynamics Model 

<img src="https://www.washington.edu/brand/files/2014/09/W-Logo_Purple_Hex.png" style="float:right;width:300px;padding:20px">   

Explore recharge and landsliding in the North Cascades National Park from a HydroShare Observatory. <br />
<br />
This Jupyter Notebook contains selected model components <br /> 
of the Watershed Dynamics model identified with a bullet  <br /> 
<br /> 
Download daily 1/16 degree gridded meteorology data, <br /> 
Download daily 1/16 degree VIC fluxes.<br /> 
Save data to a new HydroShare resource, <br /> 
Disaggreate daily data to 3-hourly, <br /> 
Estimate radiation and relative humidity from climate data with MetSim, <br /> 
Visualize daily, monthly, and annual climate data. <br /> 
* Test Recharge option in Landlab LandslideProbability component<br /> 
* Run Landlab LandslideProbability <br /> 


## HydroShare Setup and Preparation

To run this notebook, we must import several libaries.
The hs_utils library provides functions for interacting with HydroShare, including resource querying, dowloading and creation. 

In [ ]:
import os
from utilities import hydroshare #, observatory_localimport_timeseries
from datetime import datetime, timedelta
import pytz
import matplotlib.pyplot as plt
import seaborn
import numpy as np
import pandas as pd
import shutil

# set variables for interacting with HydroShare from this notebook
hs=hydroshare.hydroshare()

In [ ]:
#This is temporary until the .py is uploaded to HydroShare and Landlab utilities
# create source paths for the observatory scripts
Dest_dir = '/'.join(['/home/jovyan/work/notebooks','data',os.environ['HS_RES_ID'],os.environ['HS_RES_ID'],'data','contents'])
obs_sx_srcs = ['/'.join([Dest_dir, paths]) for paths in os.listdir(Dest_dir) if paths.startswith('landlab_') and paths.endswith('.py')]

# create the destination directory
obs_sx_dest = '/'.join(['/home/jovyan/work/notebooks','utilities'])

# copy the observatory scripts into the utilities folder
for eachsrc in obs_sx_srcs:
    shutil.copy(eachsrc, obs_sx_dest)
    print(eachsrc)

In [ ]:
# Import libraries
from utilities import landlab_landslide
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
from landlab import RasterModelGrid
from landlab.components.landslides import LandslideProbability

Next we need to establish a secure connection with HydroShare. This is done by simply instantiating the hydroshare class that is defined within hs_utils. In addition to connecting with HydroShare, this command also sets and prints environment variables for several parameters that will be useful for saving your work back to HydroShare. 

In [ ]:
# Create object to map the home directory
homedir = Dest_dir + '/'
print homedir

# if the working directory is not starting at homedir, set it to homedir
if os.getcwd() != homedir:
    os.chdir(homedir)

If you are curious about where the data is being downloaded, click on the Jupyter Notebook dashboard icon to return to the File System view.  The homedir directory location printed above is where you can find the data and contents you will download to a HydroShare JupyterHub server.  At the end of this work session, you can migrate this data to the HydroShare iRods server as a Generic Resource. 

## Create a grid on which to calculate landslide probability

In [ ]:
#Instatiate a synthetic model domain
grid = RasterModelGrid((5, 4), spacing=(0.2, 0.2))

In [ ]:
# Add required fields for the landslide component.
gridnodes = grid.number_of_nodes
grid['node']['topographic__slope'] = np.random.rand(gridnodes)
scatter_dat = np.random.randint(1, 10, gridnodes)
grid['node']['topographic__specific_contributing_area']= \
         np.sort(np.random.randint(30, 900, gridnodes))
grid['node']['soil__transmissivity']= \
         np.sort(np.random.randint(5, 20, gridnodes),-1)
grid['node']['soil__mode_total_cohesion']= \
         np.sort(np.random.randint(30, 900, gridnodes))
grid['node']['soil__minimum_total_cohesion']= \
         grid.at_node['soil__mode_total_cohesion'] - scatter_dat
grid['node']['soil__maximum_total_cohesion']= \
         grid.at_node['soil__mode_total_cohesion'] + scatter_dat
grid['node']['soil__internal_friction_angle']= \
         np.sort(np.random.randint(26, 40, gridnodes))
grid['node']['soil__thickness']= \
         np.sort(np.random.randint(1, 10, gridnodes))
grid['node']['soil__density']= \
         2000. * np.ones(grid.number_of_nodes)
         
grid_size = grid.number_of_nodes

## Number of iterations to run Monte Carlo simulation

In [ ]:
n = 10

 ## 1. Recharge Option - uniform distribution 

In [ ]:
#Option 1 - uniform distribution
distribution1 = 'uniform'
Remin_value = 20.
Remax_value = 120.

## 2. Recharge Option - Lognormal-Uniform

In [ ]:
distribution2 = 'lognormal_uniform'
Remean = 4.
Restandard_deviation = 0.25

## 3. Recharge Option - Lognormal - Spatial

In [ ]:
distribution3 = 'lognormal_spatial'
Remean3 = np.random.randint(2,7,grid_size)
Restandard_deviation3 = np.random.rand(grid_size)

## 4. Recharge Option - Data driven - Spatial 

In [ ]:
HSD_dict = {}
HSD_id_dict = {}
fract_dict = {}
# HSD id and recharge array
for vkey in range(2,8):
    HSD_dict[vkey] = np.random.randint(20,120,10)
# node id and HSD grid ids
for ckey in grid.core_nodes:
    HSD_id_dict[ckey] = np.random.randint(2,8,2)
# node id and HSD grid fraction
for ckey in grid.core_nodes:
    fract_dict[ckey] =  np.random.rand(2)
distribution4 = 'fully_distributed'
#HSD ('hydrology source domain') dictionaries
HSD_inputs = [HSD_dict,HSD_id_dict, fract_dict]


## Run and compare the Component from four Options

In [ ]:
LS_prob1 = LandslideProbability(grid,number_of_iterations=n,
    groudwater__recharge_distribution=distribution1,
    groundwater__recharge_min_value=Remin_value,
    groundwater__recharge_max_value=Remax_value)
LS_prob1.calculate_landslide_probability()
#lognormal_uniform
LS_prob2 = LandslideProbability(grid,number_of_iterations=n,
    groudwater__recharge_distribution=distribution2,
    groundwater__recharge_mean=Remean,
    groundwater__recharge_standard_deviation=Restandard_deviation)
LS_prob2.calculate_landslide_probability()
#lognormal_spatial
LS_prob3 = LandslideProbability(grid,number_of_iterations=n,
    groudwater__recharge_distribution=distribution3,
    groundwater__recharge_mean=Remean3,
    groundwater__recharge_standard_deviation=Restandard_deviation3)
LS_prob3.calculate_landslide_probability()
#HSD
LS_prob4 = LandslideProbability(grid,number_of_iterations=n,
    groudwater__recharge_distribution=distribution4,
    groudwater__recharge_HSD_inputs=HSD_inputs)
LS_prob4.calculate_landslide_probability()
